In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
nltk.download('wordnet')
nltk.download('stopwords')
#sklearn.decomposition.LatentDirichletAllocation

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Bruno\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bruno\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
data = pd.read_csv('data.csv')
data.fillna('', inplace=True)
data.head()

,contest,problem_name,problem_statement,problem_tags
0,325,A,You are given n rectangles. The corners of rec...,"implementation,*1500"
1,325,B,Daniel is organizing a football tournament. He...,"binarysearch,math,*1800"
2,325,C,Piegirl has found a monster and a book about m...,"dfsandsimilar,graphs,shortestpaths,*2600"
3,325,D,"In a far away land, there exists a planet shap...","dsu,*2900"
4,325,E,Piegirl found the red button. You have one las...,"combinatorics,dfsandsimilar,dsu,graphs,greedy,..."


In [3]:
print(f'quantidade de problemas: {data.shape[0]}')
print(f'quantidade de contests: {len(data.contest.unique())}')
print(f'problemas repetidos: {data.shape[0] - len(data.problem_statement.unique())}')

data.drop_duplicates(subset='problem_statement', inplace=True)

quantidade de problemas: 8343
quantidade de contests: 1424
problemas repetidos: 1524


## Pre-processamento

In [24]:
sw = set(stopwords.words('english'))
lemma = WordNetLemmatizer()
text = data.problem_statement

# Problem_statement
text = text.apply(lambda x: re.sub('[,|.|(|)|$|!|?|!]',' ',x)) # removendo caracteres especiais
text = text.apply(lambda x: [i for i in x.split() if i.lower() not in sw]) # removendo stop-words
text = text.apply(lambda x: ' '.join([lemma.lemmatize(i) for i in x])) # lemmatizando todo
# bigrama e trigrama todo

# Topics
topics = data.problem_tags
topics = topics.apply(lambda x: re.sub('[*][0-9]+','',x)) # remove os ratings *800 etc
topics = topics.apply(lambda x: [i for i in x.split(',') if i != '']) # tira strings vazias

In [5]:
text

0       given n rectangle corner rectangle integer coo...
1       Daniel organizing football tournament come fol...
2       Piegirl found monster book monster pie reading...
3       far away land exists planet shaped like cylind...
4       Piegirl found red button one last chance chang...
                              ...                        
8338    n block arranged row numbered left right start...
8339    map capital Berland viewed infinite coordinate...
8340    play strategic video game yeah ran good proble...
8341    first let's define function f x follows: \begi...
8342    Recently lot student enrolled Berland State Un...
Name: problem_statement, Length: 6819, dtype: object

In [25]:
topics

0                                        [implementation]
1                                    [binarysearch, math]
2                  [dfsandsimilar, graphs, shortestpaths]
3                                                   [dsu]
4       [combinatorics, dfsandsimilar, dsu, graphs, gr...
                              ...                        
8338                                       [greedy, math]
8339       [bruteforce, geometry, greedy, implementation]
8340    [datastructures, dp, greedy, implementation, s...
8341              [binarysearch, combinatorics, dp, math]
8342                                         [bruteforce]
Name: problem_tags, Length: 6819, dtype: object

## Vertorizando

In [7]:
max_df = 0.8
min_df = 0.05
vec = CountVectorizer(max_df=max_df, min_df=min_df)

In [8]:
X = vec.fit_transform(text)
X.shape

(6819, 250)

## Agrupando

In [21]:
# Pegando quantidade de topicos

total_topics = set()
topics = topics.apply(lambda x: [total_topics.add(i) for i in x])

In [22]:
n_total_topics = len(total_topics)
print(f'Número de Tópicos: {n_total_topics}')
total_topics

Número de Tópicos: 37


{'*specialproblem',
 '2-sat',
 'binarysearch',
 'bitmasks',
 'bruteforce',
 'chineseremaindertheorem',
 'combinatorics',
 'constructivealgorithms',
 'datastructures',
 'dfsandsimilar',
 'divideandconquer',
 'dp',
 'dsu',
 'expressionparsing',
 'fft',
 'flows',
 'games',
 'geometry',
 'graphmatchings',
 'graphs',
 'greedy',
 'hashing',
 'implementation',
 'interactive',
 'math',
 'matrices',
 'meet-in-the-middle',
 'numbertheory',
 'probabilities',
 'schedules',
 'shortestpaths',
 'sortings',
 'strings',
 'stringsuffixstructures',
 'ternarysearch',
 'trees',
 'twopointers'}

In [11]:
n_components = n_total_topics
max_iter = 100

lda = LatentDirichletAllocation(n_components=n_components, max_iter=max_iter)

In [12]:
lda_matrix = lda.fit_transform(X)

In [13]:
top_word = lda.components_
terms = vec.get_feature_names_out()

for i, comp in enumerate(top_word):
    top_terms_key = zip(terms, comp)
    top_terms_key= sorted(top_terms_key, key = lambda t: t[1], reverse=True)[:10]
    top_terms_list= list(dict(top_terms_key).keys())
    print("Topic "+str(i)+": ",top_terms_list)

Topic 0:  ['square', 'answer', 'correct', 'side', 'absolute', 'length', 'size', 'exceed', '10', 'considered']
Topic 1:  ['city', 'one', 'two', 'possible', 'way', 'th', 'connected', 'next', 'print', 'numbered']
Topic 2:  ['two', 'sample', 'modulo', 'different', '109', 'answer', 'single', '000', 'one', 'second']
Topic 3:  ['print', 'without', 'yes', 'otherwise', 'quote', 'no', 'two', 'second', 'possible', 'impossible']
Topic 4:  ['query', 'type', 'answer', 'one', 'two', 'th', 'next', 'second', 'following', 'guaranteed']
Topic 5:  ['time', 'second', 'moment', 'one', 'start', 'take', 'end', 'go', 'th', 'get']
Topic 6:  ['one', 'possible', 'second', 'maximum', 'minimum', 'two', 'want', 'total', 'get', 'make']
Topic 7:  ['segment', 'good', 'length', 'one', 'given', 'example', 'end', 'even', 'call', 'exactly']
Topic 8:  ['value', 'position', 'equal', 'maximum', 'let', 'calculate', 'mean', 'initial', 'consider', 'inclusive']
Topic 9:  ['tree', 'vertex', 'edge', 'given', 'path', 'two', 'next', 

## Validando

In [14]:
# todo

## Visualizando

In [15]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

panel = pyLDAvis.sklearn.prepare(lda, X, vec, mds='tsne')
panel
# todo

c:\Users\Bruno\Desktop\UNB\PIBIC\classificador-cf\venv\lib\site-packages\pyLDAvis\_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.739224  3.175303       1        1  10.680496
14    -0.988037  2.702311       2        1   8.473000
2     -0.634822  2.960003       3        1   4.861147
5     -0.593602  3.300010       4        1   4.526494
27    -1.343395  2.948804       5        1   4.381354
32    -0.681762  2.408174       6        1   4.185655
16    -0.897796  3.247712       7        1   4.016632
3     -0.785052  3.066066       8        1   3.515845
31    -0.456388  2.877227       9        1   3.411524
23    -1.118989  2.754603      10        1   3.163097
18    -0.430799  3.028910      11        1   2.798033
35    -0.349461  3.502162      12        1   2.781681
22    -0.817837  2.736075      13        1   2.716113
28    -0.555300  3.679861      14        1   2.600007
29    -0.125388  3.281928      15        1   2.517039
12    -0.343779  2.778368      16        1   2.510228
26    -1.576382  2.816291      17        1   2.460944
4     -0.423842  3.277843      18        1   2.339299
19    -1.144049  3.459156      19        1   2.194796
33    -0.963942  3.750854      20        1   2.105584
11    -0.284542  2.901921      21        1   2.054177
30    -0.884467  3.638718      22        1   1.977363
25    -1.137974  3.272911      23        1   1.939524
9     -0.972037  3.831463      24        1   1.861472
0     -0.877504  2.651133      25        1   1.809935
34    -1.731684  2.875474      26        1   1.637631
20    -1.077236  3.077815      27        1   1.533080
1     -0.661575  3.524835      28        1   1.493294
13    -0.007901  3.520027      29        1   1.448857
10    -1.450929  3.831283      30        1   1.347263
8     -1.373713  1.937382      31        1   1.294110
36    -0.087074  2.229771      32        1   1.142651
17     0.033654  4.131381      33        1   0.943737
7     -1.714787  3.277776      34        1   0.921940
15    -0.306902  4.245002      35        1   0.843472
21     0.312413  2.928926      36        1   0.757499
24    -1.315884  1.972791      37        1   0.755030, topic_info=          Term          Freq         Total Category  logprob  loglift
125         le  13482.000000  13482.000000  Default   30.000  30.0000
216     string   6975.000000   6975.000000  Default   29.000  29.0000
220       test   6519.000000   6519.000000  Default   28.000  28.0000
37        case   6392.000000   6392.000000  Default   27.000  27.0000
200   sequence   4393.000000   4393.000000  Default   26.000  26.0000
..         ...           ...           ...      ...      ...      ...
97       found      0.028086    560.616868  Topic37  -11.732  -5.0154
192  resulting      0.028086    538.325197  Topic37  -11.732  -4.9748
242    without      0.028086   1592.483791  Topic37  -11.732  -6.0594
0          000      0.028086   1409.184938  Topic37  -11.732  -5.9371
3         1000      0.028086   1168.119149  Topic37  -11.732  -5.7495

[1497 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0         1  0.293787   000
0         3  0.457002   000
0         5  0.021289   000
0         8  0.013483   000
0        13  0.091542   000
...     ...       ...   ...
249      10  0.653328  zero
249      17  0.022911  zero
249      23  0.106425  zero
249      29  0.031780  zero
249      37  0.077601  zero

[2478 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 15, 3, 6, 28, 33, 17, 4, 32, 24, 19, 36, 23, 29, 30, 13, 27, 5, 20, 34, 12, 31, 26, 10, 1, 35, 21, 2, 14, 11, 9, 37, 18, 8, 16, 22, 25])